In [5]:
import pandas as pd
import numpy as np
import pickle
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

import sys
from pathlib import Path
project_root = Path.cwd().parent
sys.path.append(str(project_root))

from src.config import *

print("✅ Kütüphaneler yüklendi")
print(f"📅 Tarih: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


✅ Kütüphaneler yüklendi
📅 Tarih: 2025-11-01 19:16:11


In [7]:
print("\n" + "="*80)
print("MODEL YÜKLEME")
print("="*80)

import joblib

# Geçici çözüm: Mevcut modellerden birini kullan
# Hangi model en iyiyse onu seç
model_name = 'gradient_boosting'  # veya gradient_boosting, decision_tree, logistic_regression

try:
    best_model = joblib.load(BEST_MODEL)
    print(f"✅ En iyi model yüklendi: {BEST_MODEL.name}")
except FileNotFoundError:
    print(f"⚠️ {BEST_MODEL} bulunamadı, {model_name} kullanılıyor...")
    best_model = joblib.load(MODELS_DIR / f"{model_name}.pkl")
    print(f"✅ Model yüklendi: {model_name}.pkl")

print(f"   Model tipi: {type(best_model).__name__}")


MODEL YÜKLEME
⚠️ c:\Users\PC\Desktop\classification_project\models\best_model.pkl bulunamadı, gradient_boosting kullanılıyor...
✅ Model yüklendi: gradient_boosting.pkl
   Model tipi: GradientBoostingClassifier


In [8]:
print("\n" + "="*80)
print("TEST VERİSİ YÜKLEME")
print("="*80)

# Test verisi
test_data = pd.read_csv(TEST_DATA_FILE)

# Özellikler
with open(PROCESSED_DATA_DIR / "feature_names.txt", 'r') as f:
    feature_names = [line.strip() for line in f.readlines()]

X_test = test_data[feature_names]
y_test = test_data[TARGET_NAME]

print(f"✅ Test verisi yüklendi")
print(f"   Satır: {len(X_test):,}")
print(f"   Özellik: {len(feature_names)}")


TEST VERİSİ YÜKLEME
✅ Test verisi yüklendi
   Satır: 1,170
   Özellik: 27


In [9]:
print("\n" + "="*80)
print("TAHMİN")
print("="*80)

# Tahmin
predictions = best_model.predict(X_test)
probabilities = best_model.predict_proba(X_test)

print(f"✅ Tahminler yapıldı")
print(f"   Phishing (-1): {(predictions == -1).sum()}")
print(f"   Legitimate (1): {(predictions == 1).sum()}")

# Doğruluk
accuracy = (predictions == y_test.values).mean()
print(f"   Doğruluk: {accuracy:.2%}")


TAHMİN
✅ Tahminler yapıldı
   Phishing (-1): 606
   Legitimate (1): 564
   Doğruluk: 95.38%


In [10]:
print("\n" + "="*80)
print("ÖRNEK TAHMİNLER")
print("="*80)

# İlk 10
results = pd.DataFrame({
    'Gerçek': y_test.values[:10],
    'Tahmin': predictions[:10],
    'Phishing %': (probabilities[:10, 0] * 100).round(1),
    'Legitimate %': (probabilities[:10, 1] * 100).round(1),
    'Doğru?': (y_test.values[:10] == predictions[:10])
})

print(results.to_string(index=False))



ÖRNEK TAHMİNLER
 Gerçek  Tahmin  Phishing %  Legitimate %  Doğru?
     -1      -1        98.9           1.1    True
     -1      -1        99.6           0.4    True
     -1      -1        99.8           0.2    True
     -1      -1        99.7           0.3    True
      1       1         6.8          93.2    True
      1       1         0.2          99.8    True
     -1      -1        99.9           0.1    True
      1       1         0.5          99.5    True
     -1       1        20.3          79.7   False
     -1      -1        98.0           2.0    True


In [11]:
print("\n" + "="*80)
print("SONUÇLARI KAYDET")
print("="*80)

# Tüm sonuçlar
all_results = pd.DataFrame({
    'Gerçek': y_test.values,
    'Tahmin': predictions,
    'Phishing_Olasılık': probabilities[:, 0],
    'Legitimate_Olasılık': probabilities[:, 1],
    'Doğru': y_test.values == predictions
})

# Kaydet
predictions_file = REPORTS_DIR / "predictions.csv"
all_results.to_csv(predictions_file, index=False)

print(f"✅ Kaydedildi: {predictions_file.name}")
print(f"   Toplam: {len(all_results)}")
print(f"   Doğru: {all_results['Doğru'].sum()}")
print(f"   Yanlış: {(~all_results['Doğru']).sum()}")



SONUÇLARI KAYDET
✅ Kaydedildi: predictions.csv
   Toplam: 1170
   Doğru: 1116
   Yanlış: 54


In [12]:
print("\n" + "="*80)
print("TEK URL TAHMİNİ")
print("="*80)

def predict_url(features_dict):
    """Tek URL tahmin et"""
    X = pd.DataFrame([features_dict])[feature_names]
    pred = best_model.predict(X)[0]
    prob = best_model.predict_proba(X)[0]
    
    label = "🚨 Phishing (Tehlikeli)" if pred == -1 else "✅ Legitimate (Güvenli)"
    confidence = prob[0] if pred == -1 else prob[1]
    
    return {
        'Sonuç': label,
        'Güven': f"{confidence*100:.1f}%",
        'Phishing': f"{prob[0]*100:.1f}%",
        'Legitimate': f"{prob[1]*100:.1f}%"
    }

# Test
sample = X_test.iloc[0].to_dict()
result = predict_url(sample)

print("🔍 Örnek Tahmin:")
for k, v in result.items():
    print(f"   {k}: {v}")



TEK URL TAHMİNİ
🔍 Örnek Tahmin:
   Sonuç: 🚨 Phishing (Tehlikeli)
   Güven: 98.9%
   Phishing: 98.9%
   Legitimate: 1.1%


In [13]:
print("\n" + "="*80)
print("PROJE ÖZETİ")
print("="*80)

summary = f"""
📊 PHISHING WEBSITE DETECTION

📊 Veri Seti:
   • Toplam: 11,055 URL
   • Train: 4,679 (80%)
   • Test: 1,170 (20%)
   • Özellik: 27

📊 Model:
   • Tip: {type(best_model).__name__}
   • Dosya: {BEST_MODEL.name}
   • Test Doğruluğu: {accuracy:.2%}

📊 Sonuçlar:
   • Phishing tespit: {(predictions == -1).sum()} URL
   • Güvenli URL: {(predictions == 1).sum()} URL
   • Tahminler: {predictions_file.name}

📊 Kullanım:
   import joblib
   model = joblib.load('best_model.pkl')
   tahmin = model.predict(X)

📅 Tarih: {datetime.now().strftime('%Y-%m-%d')}
"""

print(summary)



PROJE ÖZETİ

📊 PHISHING WEBSITE DETECTION

📊 Veri Seti:
   • Toplam: 11,055 URL
   • Train: 4,679 (80%)
   • Test: 1,170 (20%)
   • Özellik: 27

📊 Model:
   • Tip: GradientBoostingClassifier
   • Dosya: best_model.pkl
   • Test Doğruluğu: 95.38%

📊 Sonuçlar:
   • Phishing tespit: 606 URL
   • Güvenli URL: 564 URL
   • Tahminler: predictions.csv

📊 Kullanım:
   import joblib
   model = joblib.load('best_model.pkl')
   tahmin = model.predict(X)

📅 Tarih: 2025-11-01



In [14]:
print("\n" + "="*80)
print("MODEL PAKETİ OLUŞTUR")
print("="*80)

# Paket oluştur
model_package = {
    'model': best_model,
    'feature_names': feature_names,
    'accuracy': accuracy,
    'metadata': {
        'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'model_type': type(best_model).__name__,
        'test_size': len(X_test),
        'features': len(feature_names)
    }
}

# Kaydet
package_path = MODELS_DIR / "model_package.pkl"
with open(package_path, 'wb') as f:
    pickle.dump(model_package, f)

print(f"✅ Model paketi: {package_path.name}")


MODEL PAKETİ OLUŞTUR
✅ Model paketi: model_package.pkl


In [15]:
print("\n" + "="*80)
print("✅ DEPLOYMENT TAMAMLANDI!")
print("="*80)

checklist = """
✅ TAMAMLANAN İŞLER:
   ✓ Model yüklendi
   ✓ Test verisi yüklendi
   ✓ Tahminler yapıldı
   ✓ Sonuçlar kaydedildi
   ✓ Tek tahmin fonksiyonu
   ✓ Model paketi oluşturuldu

📁 DOSYALAR:
   ✓ models/best_model.pkl
   ✓ models/model_package.pkl
   ✓ reports/predictions.csv
   ✓ data/processed/feature_names.txt

🚀 SONRAKİ ADIMLAR:
   • Web uygulaması (Streamlit)
   • API (FastAPI)
   • Docker container
   • Cloud deploy
"""

print(checklist)


✅ DEPLOYMENT TAMAMLANDI!

✅ TAMAMLANAN İŞLER:
   ✓ Model yüklendi
   ✓ Test verisi yüklendi
   ✓ Tahminler yapıldı
   ✓ Sonuçlar kaydedildi
   ✓ Tek tahmin fonksiyonu
   ✓ Model paketi oluşturuldu

📁 DOSYALAR:
   ✓ models/best_model.pkl
   ✓ models/model_package.pkl
   ✓ reports/predictions.csv
   ✓ data/processed/feature_names.txt

🚀 SONRAKİ ADIMLAR:
   • Web uygulaması (Streamlit)
   • API (FastAPI)
   • Docker container
   • Cloud deploy

